In [1]:
%pylab
%matplotlib inline

import cv2, re, isbi_seg_oak
from segmetrics.study    import *
from segmetrics.regional import *
from segmetrics.boundary import *
from skimage import measure

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
result      = cv2.imread('testdata/result.png', 0)
groundtruth = cv2.imread('testdata/groundtruth.png', 0)

In [3]:
%%timeit
study = Study()
study.add_measure(ISBIScore())
study.set_expected(groundtruth)
study.process(result)

1 loop, best of 3: 1.06 s per loop


In [4]:
n, study = 5, Study()

study.add_measure(Dice()     , 'dice')
study.add_measure(Hausdorff(), 'hausdorff')
study.add_measure(ISBIScore(), 'isbi score')

study.set_expected(groundtruth)
for i in xrange(n): study.process(result)

for measure_name in study.results:
    print('%10s: %6.3f ±%6.3f' % (measure_name, mean(study.results[measure_name]), std(study.results[measure_name])))

 hausdorff: 16.480 ±27.855
isbi score:  0.752 ± 0.237
      dice:  0.916 ± 0.000


In [5]:
oak_result = isbi_seg_oak.oak(groundtruth, result, n)
match = re.compile(r'.*?([\.0-9]+)').match(oak_result)
if match is not None:
    official_result = float(match.group(1))
    print('Error:', official_result - mean(study.results['isbi score']))
else:
    raise ValueError('Unexpected result:', oak_result)

Error: -1.99170556492e-07


In [6]:
study2 = Study()
for measure in study.measures.items():
    study2.add_measure(*measure[::-1])
study2.set_expected(groundtruth, unique=False)
for i in xrange(n): study2.process(result)
for measure_name in study.measures.keys():
    assert allclose(study.results[measure_name], study2.results[measure_name])